# Dummy Example

This notebook shows the use of the dummy feedback function provider which
behaves like the huggingface provider except it does not actually perform any
network calls and just produces constant results. It can be used to prototype
feedback function wiring for your apps before invoking potentially slow (to
run/to load) feedback functions.

In [ ]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import sys

# If running from github repo, can use this:
sys.path.append(str(Path().cwd().parent.resolve()))

In [ ]:
import threading

from examples.frameworks.custom.custom_app import CustomApp

from trulens_eval import Feedback
from trulens_eval import FeedbackMode
from trulens_eval import Tru
from trulens_eval.feedback.provider.hugs import Dummy
from trulens_eval.feedback.provider.hugs import Huggingface
from trulens_eval.tru_custom_app import TruCustomApp
from trulens_eval.utils.threading import TP

tru = Tru()

tru.reset_database()

tru.start_dashboard(
    force = True,
    _dev=Path().cwd().parent.resolve()
)

In [ ]:
# hugs = Huggingface()
hugs = Dummy()

f_positive_sentiment = Feedback(hugs.positive_sentiment).on_output()

In [ ]:
# Test of a sequence of records:
"""
with ta:
    for i, q in enumerate(["hello there"] * 100):
        print(f"\rrequest {i} ", end="")
        print(f"thread count={threading.active_count()}, promises={TP().promises.qsize()}", end="")

        res = ca.respond_to_query(input=q)
"""

In [ ]:
from ipywidgets import interact
from ipywidgets import widgets
from IPython.display import JSON
from trulens_eval.utils.serial import JSONPath
from trulens_eval.app import App
from typing import Any, Callable
from traitlets import Unicode, Bool, validate, TraitError, HasTraits
import traitlets
from ipywidgets import DOMWidget, register
from typing import Mapping, Sequence
import pydantic
from trulens_eval.utils.json import JSON_BASES, jsonify
from threading import Thread

import asyncio

from pprint import PrettyPrinter
pp = PrettyPrinter()

debug_style = dict(border="0px solid gray", padding="0px")

class Selector(HasTraits):
    select = Unicode()
    jpath = traitlets.Any()

    def __init__(self, select: str, make_on_delete: Callable):
        self.select = select
        self.jpath = JSONPath.of_string(select)

        self.w_edit = widgets.Text(value=select, layout=debug_style)
        self.w_delete = widgets.Button(description="x", layout=dict(width="30px", **debug_style))

        self.on_delete = make_on_delete(self)
        self.w_delete.on_click(self.on_delete)

        traitlets.link((self.w_edit, "value"), (self, "select"))

        def on_update_select(ev):
            try:
                jpath = JSONPath.of_string(ev.new)
                self.jpath = jpath
            except Exception:
                pass
            
        self.observe(on_update_select, ["select"])

        self.w = widgets.HBox([self.w_delete, self.w_edit], layout=debug_style)

    
class SelectorValue(HasTraits):
    selector = traitlets.Any()
    obj = traitlets.Any()

    def __init__(self, selector: Selector):
        self.selector = selector
        self.obj = None

        self.w_listing = widgets.HTML(layout=debug_style)
        self.w = widgets.VBox([self.selector.w, self.w_listing], layout=debug_style)

        self.selector.observe(self.update_selector, "jpath")
        self.observe(self.update_obj, "obj")

    def update_selector(self, ev):
        self.update()

    def update_obj(self, ev):
        self.update()

    def update(self):
        obj = self.obj
        jpath = self.selector.jpath

        inner_obj = None
        inner_class = None

        if obj is None:
            ret_html = "no listing yet"
        else:
            try:
                inner_obj = next(iter(jpath(obj)))
                inner_class = type(inner_obj)

                inner_obj = jsonify(inner_obj)

                # if isinstance(inner_obj, pydantic.BaseModel):
                #    inner_obj = inner_obj.dict()

                if isinstance(inner_obj, JSON_BASES):
                    ret_html = str(inner_obj)[0:128]

                elif isinstance(inner_obj, Mapping):
                    ret_html = "<ul>"
                    for key, val in inner_obj.items():
                        ret_html += f"<li>{key} = {str(val)[0:32]}</li>"
                    ret_html += "</ul>"

                elif isinstance(inner_obj, Sequence):
                    ret_html = "<ul>"
                    for i, val in enumerate(inner_obj):
                        ret_html += f"<li>[{i}] = {str(val)[0:32]}</li>"
                    ret_html += "</ul>"                    

                else:
                    ret_html = str(inner_obj)[0:32]

            except Exception as e:
                self.w_listing.layout.border = "1px solid red"
                return

        self.w_listing.layout.border = "0px solid black"
        self.w_listing.value = f"<div>({inner_class.__name__} as {type(inner_obj).__name__}): {ret_html}</div>"


class RecordWiget():
    def __init__(self, record_selections, record=None, human_or_input=None):
        self.record = record
        self.record_selections = record_selections
        self.record_values = dict()

        self.human_or_input = widgets.HBox([human_or_input], layout=debug_style)
        self.w_human = widgets.HBox([widgets.HTML("<b>human:</b>"), self.human_or_input], layout=debug_style)
        self.d_comp = widgets.HTML(layout=debug_style)
        self.d_extras = widgets.VBox(layout=debug_style)

        self.human = ""
        self.comp = ""

        self.d = widgets.VBox([self.w_human, self.d_comp, self.d_extras], layout={**debug_style, "border": "5px solid #aaaaaa"})

    def update_selections(self):
        # change to trait observe
        for s in self.record_selections:
            if s not in self.record_values:
                sv = SelectorValue(selector=s)
                self.record_values[s] = sv
                self.d_extras.children += (sv.w, )

            if self.record is not None:
                record_filled = self.record.layout_calls_as_app()
            else:
                record_filled = None

            self.record_values[s].obj =  record_filled

    def remove_selector(self, selector: Selector):
        if selector not in self.record_values:
            return
            
        item = self.record_values[selector]
        del self.record_values[selector]
        new_children = list(self.d_extras.children)
        new_children.remove(item.w)
        self.d_extras.children = tuple(new_children)

    def set_human(self, human: str):
        self.human = human
        self.human_or_input.children = (widgets.HTML(f"<div>{human}</div>", layout=debug_style), )

    def set_comp(self, comp: str):
        self.comp = comp
        self.d_comp.value = f"<div><b>computer:</b> {comp}</div>"

class AppUI(traitlets.HasTraits):
    # very prototype

    def __init__(self, app: App, use_async: bool = False):
        self.use_async = use_async

        self.app = app

        self.main_input = widgets.Text(layout=debug_style)
        self.app_selector = widgets.Text(layout=debug_style)
        self.record_selector = widgets.Text(layout=debug_style)

        self.main_input_button = widgets.Button(description="+ Record", layout=debug_style)
        self.app_selector_button = widgets.Button(description="+ Select.App", layout=debug_style)
        self.record_selector_button = widgets.Button(description="+ Select.Record", layout=debug_style)

        self.display_top = widgets.VBox([], layout=debug_style)
        self.display_side = widgets.VBox([], layout={'min_width':"500px", **debug_style})

        self.display_records = []

        self.app_selections = {}
        self.record_selections = []

        self.current_record = RecordWiget(record_selections=self.record_selections, human_or_input=self.main_input)
        self.current_record_record = None

        self.records = [self.current_record]

        self.main_input.on_submit(self.add_record)
        self.app_selector.on_submit(self.add_app_selection)
        self.record_selector.on_submit(self.add_record_selection)

        self.main_input_button.on_click(self.add_record)
        self.app_selector_button.on_click(self.add_app_selection)
        self.record_selector_button.on_click(self.add_record_selection)

        self.display_bottom = widgets.VBox([
            widgets.HBox([self.main_input_button, self.main_input], layout=debug_style),
            widgets.HBox([self.app_selector_button, self.app_selector], layout=debug_style),
            widgets.HBox([self.record_selector_button, self.record_selector], layout=debug_style)],
            layout=debug_style
        )

        self.display_top.children += (self.current_record.d, )

        self.d = widgets.HBox([
                widgets.VBox([
                    self.display_top,
                    self.display_bottom
                ], layout=debug_style),
                self.display_side], layout=debug_style
            )

    def make_on_delete_record_selector(self, selector):
        def on_delete(ev):
            print("delete record selector", str(selector.jpath))

            self.record_selections.remove(selector)

            for r in self.records:
                r.remove_selector(selector)

        return on_delete

    def make_on_delete_app_selector(self, selector):
        def on_delete(ev):
            print("delete app selector", str(selector.jpath))

            sw = self.app_selections[selector]
            del self.app_selections[selector]

            new_children = list(self.display_side.children)
            new_children.remove(sw.w)

            self.display_side.children = tuple(new_children)

        return on_delete

    def update_app_selections(self):
        for _, sw in self.app_selections.items():
            sw.update()

    def add_app_selection(self, w):
        s = self.app_selector.value

        sel = Selector(select=s, make_on_delete=self.make_on_delete_app_selector)

        sw = SelectorValue(selector=sel)
        self.app_selections[sel] = sw
        sw.obj = self.app

        self.display_side.children += (sw.w, )

    def add_record_selection(self, w):
        s = self.record_selector.value

        sel = Selector(select=s, make_on_delete=self.make_on_delete_record_selector)
        self.record_selections.append(sel)

        for r in self.records:
            r.update_selections()

    def add_record(self, w):
        human = self.main_input.value

        if len(human) == 0:
            return

        self.current_record.set_human(human)

        with self.app as recording:
            # generalize
            if self.use_async:
                self.current_record.set_comp("generating:")

                comp = ""

                def run_in_thread(comp):
                    async def run_in_main_loop(comp):
                        comp_generator = await self.app.main_acall(human)
                        async for tok in comp_generator:
                            comp += tok
                            self.current_record.set_comp(comp)

                    loop = asyncio.new_event_loop()
                    asyncio.set_event_loop(loop)
                    loop.run_until_complete(asyncio.Task(run_in_main_loop(comp)))

                t = Thread(target=run_in_thread, args=(comp, ))
                t.start()
                t.join()

            else:
                self.current_record.set_comp("...")
                comp = self.app.main_call(human)
                self.current_record.set_comp(comp)

        self.current_record_record = recording.get()
        self.current_record.record = self.current_record_record
        self.current_record.update_selections()

        self.update_app_selections()

        self.current_record = RecordWiget(record_selections=self.record_selections, human_or_input=self.main_input)
        self.records.append(self.current_record)
        self.display_top.children += (self.current_record.d, )

In [ ]:
# Create custom app:
ca = CustomApp()

# Create trulens wrapper:
ta = TruCustomApp(
    ca,
    app_id="customapp",
    # feedback_mode=FeedbackMode.WITH_APP
    feedbacks=[f_positive_sentiment]
)

# Must be set after above constructor as the instrumentation changes what
# CustomApp.respond_to_query points to.
ta.main_method = CustomApp.respond_to_query
ta.main_async_method = CustomApp.arespond_to_query

aui = AppUI(app=ta, use_async=True)
aui.d

In [ ]:
# ca.arespond_to_query("hello")
#async for tok in ca.arespond_to_query("hello"):
#    print(tok, end="")

In [ ]:
"""
generator = await ta.main_acall("hello")
async for tok in generator:
    print(tok, end="")
"""

In [ ]:
#async for tok in temp:
#    print(tok)

In [ ]:
# ta.print_instrumented()

In [ ]:
from asyncio import sleep
import asyncio

async def arespond_to_query(input):
    # fake async call, must return an async token generator and final result

    res = f"this is the fake output to {input}: {input} again"

    ret = ""

    async def async_generator():
        nonlocal ret
        for tok in res.split(" "):
            await sleep(0.5)
            yield tok + " "
            ret += tok + " "

    it = async_generator()

    async def collect_gen():
        nonlocal ret

        async for tok in it:
            # finish if not done yet
            ret += tok

        return ret
    
    return it, collect_gen()

In [ ]:
gen, res = await arespond_to_query("hello there")

async for tok in gen:
    print(tok, end="")

final_res = await res
print("\nfinal=", final_res)

In [ ]:
await res()